
Author: Amparo Godoy Pastore <br>
Date: 1/22/2023

# Introduction

In this project I use public hurricane forecast data from the NHC to create forecast visualizations of hurricane Nicole and perform basic data analysis tasks. I plot the forecasted paths, as well as the true track of the hurricane using interpolation.

This involves:

- Importing forecast data from the NHC and grouping it in lists to create visualizations and perform data analysis.
- Plotting the true track and the forecasts of the hurricane on a map.
- Interpolating the hurricane's path.
- Performing basic data analysis tasks.

# 1. Preliminaries

In [9]:
import pickle                           # Load and save pickle files.
import datetime                         # Handle date/time objects.
import re                               # Regular expressions.
import matplotlib as mpl                # Plot things.
import folium                           # Plot interactive maps.
import numpy as np                      # Basic math and array operations.
import scipy as sp                      # More math and array operations.

# Import the forecast data.
with open('forecasts_2022_nicole.pkl','rb') as f:
    forecasts = pickle.load(f)

Here I create three lists called `true_times`,`true_latitudes`, and `true_longitudes` that contain the times, latitudes, and longitudes of each point along the true track. I also create a list called `true_points` containing (latitude,longitude) pairs along the true track.


In [10]:
#Empty lists for true track
true_times = []
true_latitudes = [] 
true_longitudes = []

#Loop over forecasts and append elements to lists
for i in forecasts:
    true_times.append(i[0][0])
    true_latitudes.append(i[0][1])
    true_longitudes.append(i[0][2])

#Creates list of true points
true_points = list(zip(true_latitudes, true_longitudes))

# 2. Plotting on a map

The following map shows all the forecasts in different colors and the true track of the hurricane as a thick black line.

In [11]:
#Create list of colors for plotting forecasts
colors = mpl.cm.jet(np.linspace(0,1,len(forecasts)))
colors = [mpl.colors.rgb2hex(c) for c in colors]

# Create the base map.
m = folium.Map()

#Add forecasts to the map
for i in range(len(forecasts)):
    forecast_latitudes = []
    forecast_longitudes = []
    for x in forecasts[i]:
        forecast_latitudes.append(x[1])
        forecast_longitudes.append(x[2])
        forecast_points = list(zip(forecast_latitudes, forecast_longitudes))
    folium.PolyLine(forecast_points, color = colors[i], weight = 1, opacity = 1).add_to(m) #Plots lines

#Add true path to the map
folium.PolyLine(true_points, color = "black", weight = 4, opacity = 1).add_to(m)

#Resize map to fit data
m.fit_bounds(m.get_bounds(), padding = (10, 10))
    
#Display map
display(m)

# 3. Interpolating the hurricane's path

Next, I create a list containing the number of seconds elapsed between the first forecast and every forecast (including the first forecast) to perform interpolation.

In [12]:
#Create list containing number of secs between first forecast and every other forecast
true_times_sec = []
for i in range(len(true_times)):
    dt = true_times[i] - true_times[0]
    true_times_sec.append(dt.total_seconds()) 

Here, I define functions that return the approximate actual location of the hurricane `t` seconds after the first forecast.

In [13]:
from scipy import interpolate

#Defining function that takes a number of seconds t as an argument and returns the approximate longitude of the hurricane
def longitude(t):
    interp = interpolate.interp1d(true_times_sec, true_longitudes)
    return interp(t)

#Same thing, but this one returns the approximate latitude of the hurricane
def latitude(t):
    interp = interpolate.interp1d(true_times_sec, true_latitudes)
    return interp(t)

I now define a second version of the longitude function that does not use `scipy.interpolate.interp1d`, and I test it against the one I defined above.

In [14]:
#Writing the longitude function without scipy.interpolate.interp1d
def longitude2(t):
    for i in range(len(true_times_sec)):
        if t>true_times_sec[i] and t<true_times_sec[i+1]: #Locating the x coordinates of the two points
            x = i #Storing index
    #Formula for interpolation
    interp = true_longitudes[x]+((t-true_times_sec[x])*((true_longitudes[x+1]-true_longitudes[x])/(true_times_sec[x+1]-true_times_sec[x])))
    return interp

#Testing against interp1d function
t = 3000
print("Estimated longitude at {} seconds - with scipy: ".format(t), longitude(t))
print("Estimated longitude at {} seconds - without scipy: ".format(t), longitude2(t))

Estimated longitude at 3000 seconds - with scipy:  -68.325
Estimated longitude at 3000 seconds - without scipy:  -68.325


Using the interpolation above, I replot the true path with 200 more points. The interpolated points, shown in blue, are consistent with the expected linear interpolation.

In [15]:
# Create the base map.
m = folium.Map()

# Add circular marker
for i in true_points:
    folium.CircleMarker(i, fill = True, color = "black", fill_opacity = 1, radius = 1, popup = 'Pop up text').add_to(m)
    
#Add true path to the map
folium.PolyLine(true_points, color = "black", weight = 2, opacity = 1).add_to(m)

#Create a list with times in seconds
secs = np.linspace(0, true_times_sec[len(true_times_sec)-1], 200)

#Use latitude and longitude functions to create a new list of interpolated points
for t in secs:
    interp_points = list((latitude(t), longitude(t)))
    folium.CircleMarker(interp_points, fill = True, color = "blue", fill_opacity = 1, radius = 1, popup = 'Pop up text').add_to(m)

#Resize map to fit data
m.fit_bounds(m.get_bounds(), padding = (10, 10))

#Display
display(m)

# Conclusion

The main goal of this notebook was to device an efficient visualization for the true track and the forecasts of hurricane Nicole. By retrieving and strategically organizing public data from the NHC website, I was able to plot the true path and forecasts of the hurricane in a map. To make the data more accesible and understandable, I gave distinct colors to each track. Furthermore, I interpolated the hurricane's true path and replotted it on the map with 200 more points. This allowed me to reconstruct the full path from a small discrete set of points to a more accurate representation. Overall, the tasks performed in this notebook contributed to an informative graphical representation of numerical data, and provided an accessible way to quantify the hurricane's distance to campus or the forecasting error.